In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
offertimes.to_csv('offertimes.csv')
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
inc_prod_num.to_csv('inc_prod_num.csv')
choice_prob = raw_jd_choice['clicknum'] / raw_jd_choice.groupby('clickset')['clicknum'].transform('sum')
raw_jd_choice['choice prob'] = choice_prob

assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

# all purchase options in each assortment
purchaseset = raw_jd_choice['purchaseset']
purchaseset_list = []
for pset in purchaseset:
    num_lst = json.loads(pset)
    purchaseset_list.append(num_lst)
raw_jd_choice['purchaseset'] = purchaseset_list

purchasenum_list = raw_jd_choice['clicknum'].tolist()
jd_choice=pd.DataFrame({'clickset':clickset_list,'purchaseset':purchaseset_list,'purchasenum':purchasenum_list,'choice prob':choice_prob})

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
collection = []
for cset in clickset_list:
    if cset not in collection:
        collection.append(cset)
print('there are {} different assortments'.format(len(collection)))

assortment_info_df['assortments'] = collection
choice_collection = np.zeros((n,len(collection)))
global_num = 0
for i in range(len(collection)):
    local_num =0
    while(local_num < inc_prod_num[i]):
        if purchaseset_list[global_num][0] in collection[i]:
            choice_collection[purchaseset_list[global_num][0]][i] = choice_prob[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print('error!')
full_choice_collection = pd.DataFrame(choice_collection)
full_choice_collection.to_csv('instances/full_instances/full_choice.csv')


there are 9 different products
there are 134 different assortments


In [4]:
assortment_info_df_20 = assortment_info_df[(assortment_info_df['offer_times']>=20)]
assortments_20 = []
for i in range(len(assortment_info_df_20['assortments'])):
    assortments_20.append(assortment_info_df_20['assortments'][i])


jd_choice_20 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_20:
        jd_choice_20.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_20 = assortment_info_df_20['assortments']
choice_collection_20 = np.zeros((n,len(collection_20)))

frequency_collection_20 = np.zeros((n,len(collection_20)))

purchaseset_list_20 = jd_choice_20['purchaseset'].tolist()

inc_prod_num_20 = assortment_info_df_20['includ_prod_num']
choice_prob_20 = jd_choice_20['choice prob']
frequency_20 = jd_choice_20['purchasenum']

global_num = 0
for i in range(len(collection_20)):
    local_num =0
    while(local_num < inc_prod_num_20[i]):
        if purchaseset_list_20[global_num][0] in assortments_20[i]:
            choice_collection_20[purchaseset_list_20[global_num][0]][i] = choice_prob_20[global_num]
            frequency_collection_20[purchaseset_list_20[global_num][0]][i] = frequency_20[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_20[global_num])
            print(assortments_20[i])
            print('error!')
            break
#print(global_num)  

choice_check_20 = pd.DataFrame(choice_collection_20)
choice_check_20.to_csv('instances/full_instances/full_20.csv')
frequency_collection_20 = frequency_collection_20.astype(int)
frequency_check_20 = pd.DataFrame(frequency_collection_20)
frequency_check_20.to_csv('instances/full_instances/frequency_20.csv')

offer_times_20 = assortment_info_df_20['offer_times'].tolist()
print('the number of assortments with at least 20 purchases is',len(assortments_20))

the number of assortments with at least 20 purchases is 29


In [5]:
assortment_info_df_30 = assortment_info_df[(assortment_info_df['offer_times']>=30)]
assortments_30 = []
for i in range(len(assortment_info_df_30['assortments'])):
    assortments_30.append(assortment_info_df_30['assortments'][i])


jd_choice_30 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_30:
        jd_choice_30.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_30 = assortment_info_df_30['assortments']
choice_collection_30 = np.zeros((n,len(collection_30)))

frequency_collection_30 = np.zeros((n,len(collection_30)))

purchaseset_list_30 = jd_choice_30['purchaseset'].tolist()

inc_prod_num_30 = assortment_info_df_30['includ_prod_num']
choice_prob_30 = jd_choice_30['choice prob']
frequency_30 = jd_choice_30['purchasenum']

global_num = 0
for i in range(len(collection_30)):
    local_num =0
    while(local_num < inc_prod_num_30[i]):
        if purchaseset_list_30[global_num][0] in assortments_30[i]:
            choice_collection_30[purchaseset_list_30[global_num][0]][i] = choice_prob_30[global_num]
            frequency_collection_30[purchaseset_list_30[global_num][0]][i] = frequency_30[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_30[global_num])
            print(assortments_30[i])
            print('error!')
            break
#print(global_num)  

choice_check_30 = pd.DataFrame(choice_collection_30)
choice_check_30.to_csv('instances/full_instances/full_30.csv')
frequency_collection_30 = frequency_collection_30.astype(int)
frequency_check_30 = pd.DataFrame(frequency_collection_30)
frequency_check_30.to_csv('instances/full_instances/frequency_30.csv')

offer_times_30 = assortment_info_df_30['offer_times'].tolist()
print('the number of assortments with at least 30 purchases is',len(assortments_30))

the number of assortments with at least 30 purchases is 24


In [6]:
assortment_info_df_40 = assortment_info_df[(assortment_info_df['offer_times']>=40)]
assortments_40 = []
for i in range(len(assortment_info_df_40['assortments'])):
    assortments_40.append(assortment_info_df_40['assortments'][i])


jd_choice_40 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_40:
        jd_choice_40.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_40 = assortment_info_df_40['assortments']
choice_collection_40 = np.zeros((n,len(collection_40)))

frequency_collection_40 = np.zeros((n,len(collection_40)))

purchaseset_list_40 = jd_choice_40['purchaseset'].tolist()

inc_prod_num_40 = assortment_info_df_40['includ_prod_num']
choice_prob_40 = jd_choice_40['choice prob']
frequency_40 = jd_choice_40['purchasenum']

global_num = 0
for i in range(len(collection_40)):
    local_num =0
    while(local_num < inc_prod_num_40[i]):
        if purchaseset_list_40[global_num][0] in assortments_40[i]:
            choice_collection_40[purchaseset_list_40[global_num][0]][i] = choice_prob_40[global_num]
            frequency_collection_40[purchaseset_list_40[global_num][0]][i] = frequency_40[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_40[global_num])
            print(assortments_40[i])
            print('error!')
            break
#print(global_num)  

choice_check_40 = pd.DataFrame(choice_collection_40)
choice_check_40.to_csv('instances/full_instances/full_40.csv')
frequency_collection_40 = frequency_collection_40.astype(int)
frequency_check_40 = pd.DataFrame(frequency_collection_40)
frequency_check_40.to_csv('instances/full_instances/frequency_40.csv')

offer_times_40 = assortment_info_df_40['offer_times'].tolist()
print('the number of assortments with at least 40 purchases is',len(assortments_40))

the number of assortments with at least 40 purchases is 19


In [7]:
assortment_info_df_50 = assortment_info_df[(assortment_info_df['offer_times']>=50)]
assortments_50 = []
for i in range(len(assortment_info_df_50['assortments'])):
    assortments_50.append(assortment_info_df_50['assortments'][i])


jd_choice_50 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_50:
        jd_choice_50.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_50 = assortment_info_df_50['assortments']
choice_collection_50 = np.zeros((n,len(collection_50)))

frequency_collection_50 = np.zeros((n,len(collection_50)))

purchaseset_list_50 = jd_choice_50['purchaseset'].tolist()

inc_prod_num_50 = assortment_info_df_50['includ_prod_num']
choice_prob_50 = jd_choice_50['choice prob']
frequency_50 = jd_choice_50['purchasenum']

global_num = 0
for i in range(len(collection_50)):
    local_num =0
    while(local_num < inc_prod_num_50[i]):
        if purchaseset_list_50[global_num][0] in assortments_50[i]:
            choice_collection_50[purchaseset_list_50[global_num][0]][i] = choice_prob_50[global_num]
            frequency_collection_50[purchaseset_list_50[global_num][0]][i] = frequency_50[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_50[global_num])
            print(assortments_50[i])
            print('error!')
            break
#print(global_num)  

choice_check_50 = pd.DataFrame(choice_collection_50)
choice_check_50.to_csv('instances/full_instances/full_50.csv')
frequency_collection_50 = frequency_collection_50.astype(int)
frequency_check_50 = pd.DataFrame(frequency_collection_50)
frequency_check_50.to_csv('instances/full_instances/frequency_50.csv')

offer_times_50 = assortment_info_df_50['offer_times'].tolist()
print('the number of assortments with at least 50 purchases is',len(assortments_50))

the number of assortments with at least 50 purchases is 15


In [8]:
assortment_info_df_60 = assortment_info_df[(assortment_info_df['offer_times']>=60)]
assortments_60 = []
for i in range(len(assortment_info_df_60['assortments'])):
    assortments_60.append(assortment_info_df_60['assortments'][i])


jd_choice_60 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_60:
        jd_choice_60.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_60 = assortment_info_df_60['assortments']
choice_collection_60 = np.zeros((n,len(collection_60)))

frequency_collection_60 = np.zeros((n,len(collection_60)))

purchaseset_list_60 = jd_choice_60['purchaseset'].tolist()

inc_prod_num_60 = assortment_info_df_60['includ_prod_num']
choice_prob_60 = jd_choice_60['choice prob']
frequency_60 = jd_choice_60['purchasenum']

global_num = 0
for i in range(len(collection_60)):
    local_num =0
    while(local_num < inc_prod_num_60[i]):
        if purchaseset_list_60[global_num][0] in assortments_60[i]:
            choice_collection_60[purchaseset_list_60[global_num][0]][i] = choice_prob_60[global_num]
            frequency_collection_60[purchaseset_list_60[global_num][0]][i] = frequency_60[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_60[global_num])
            print(assortments_60[i])
            print('error!')
            break
#print(global_num)  

choice_check_60 = pd.DataFrame(choice_collection_60)
choice_check_60.to_csv('instances/full_instances/full_60.csv')
frequency_collection_60 = frequency_collection_60.astype(int)
frequency_check_60 = pd.DataFrame(frequency_collection_60)
frequency_check_60.to_csv('instances/full_instances/frequency_60.csv')

offer_times_60 = assortment_info_df_60['offer_times'].tolist()
print('the number of assortments with at least 60 purchases is',len(assortments_60))

the number of assortments with at least 60 purchases is 13


In [9]:
assortment_info_df_75 = assortment_info_df[(assortment_info_df['offer_times']>=75)]
assortments_75 = []
for i in range(len(assortment_info_df_75['assortments'])):
    assortments_75.append(assortment_info_df_75['assortments'][i])


jd_choice_75 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_75:
        jd_choice_75.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_75 = assortment_info_df_75['assortments']
choice_collection_75 = np.zeros((n,len(collection_75)))

frequency_collection_75 = np.zeros((n,len(collection_75)))

purchaseset_list_75 = jd_choice_75['purchaseset'].tolist()

inc_prod_num_75 = assortment_info_df_75['includ_prod_num']
choice_prob_75 = jd_choice_75['choice prob']
frequency_75 = jd_choice_75['purchasenum']

global_num = 0
for i in range(len(collection_75)):
    local_num =0
    while(local_num < inc_prod_num_75[i]):
        if purchaseset_list_75[global_num][0] in assortments_75[i]:
            choice_collection_75[purchaseset_list_75[global_num][0]][i] = choice_prob_75[global_num]
            frequency_collection_75[purchaseset_list_75[global_num][0]][i] = frequency_75[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_75[global_num])
            print(assortments_75[i])
            print('error!')
            break
#print(global_num)  

choice_check_75 = pd.DataFrame(choice_collection_75)
choice_check_75.to_csv('instances/full_instances/full_75.csv')
frequency_collection_75 = frequency_collection_75.astype(int)
frequency_check_75 = pd.DataFrame(frequency_collection_75)
frequency_check_75.to_csv('instances/full_instances/frequency_75.csv')

offer_times_75 = assortment_info_df_75['offer_times'].tolist()
print('the number of assortments with at least 75 purchases is',len(assortments_75))

the number of assortments with at least 75 purchases is 12


In [10]:
assortment_info_df_100 = assortment_info_df[(assortment_info_df['offer_times']>=100)]
assortments_100 = []
for i in range(len(assortment_info_df_100['assortments'])):
    assortments_100.append(assortment_info_df_100['assortments'][i])

jd_choice_100 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_100:
        jd_choice_100.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_100 = assortment_info_df_100['assortments']
choice_collection_100 = np.zeros((n,len(collection_100)))

frequency_collection_100 = np.zeros((n,len(collection_100)))

purchaseset_list_100 = jd_choice_100['purchaseset'].tolist()

inc_prod_num_100 = assortment_info_df_100['includ_prod_num']
choice_prob_100 = jd_choice_100['choice prob']
frequency_100 = jd_choice_100['purchasenum']

global_num=0
for i in range(len(collection_100)):
    local_num =0
    while(local_num < inc_prod_num_100[i]):
        if purchaseset_list_100[global_num][0] in assortments_100[i]:
            choice_collection_100[purchaseset_list_100[global_num][0]][i] = choice_prob_100[global_num]
            frequency_collection_100[purchaseset_list_100[global_num][0]][i] = frequency_100[global_num]
            global_num = global_num +1
            local_num = local_num +1

choice_check_100 = pd.DataFrame(choice_collection_100)
choice_check_100.to_csv('instances/full_instances/full_100.csv')
frequency_collection_100 = frequency_collection_100.astype(int)
frequency_check_100 = pd.DataFrame(frequency_collection_100)
frequency_check_100.to_csv('instances/full_instances/frequency_100.csv')

offer_times_100 = assortment_info_df_100['offer_times'].tolist()
print('the number of assortments with at least 100 purchases is',len(assortments_100))

the number of assortments with at least 100 purchases is 11


representability: even for the data set with assortments offered with 100 times, such a data set cannot be representable for the regular model, RUM and MDM cannot represent all the instances above

In [11]:
collection_100 = collection_100.tolist()
''' check regularity '''
flag = 0
for i in range(len(collection_100)):
    for j in range(i+1,len(collection_100)):
        if {*collection_100[i]} <= {*collection_100[j]}:
            for k in collection_100[i]:
                #if k !=0 and choice_collection_100[k][i] < choice_collection_100[k][j]:
                if choice_collection_100[k][i] < choice_collection_100[k][j]:
                    flag = 1
        if {*collection_100[i]} >= {*collection_100[j]}:
            for k in collection_100[j]:
                if choice_collection_100[k][j] < choice_collection_100[k][i]:
                #if k !=0 and choice_collection_100[k][j] < choice_collection_100[k][i]:
                    flag = 1
    if flag == 1:
        break
if flag == 0:
    print('REGULAR')
else:
    print('NOT REGURLAR')

NOT REGURLAR


In [12]:
import pyomo.environ as pyo
from pyomo.environ import *

def mccm_limit(data, collection, offertimes):
    #print(f'check the offertimes {offertimes}')
    model = pyo.ConcreteModel(name="mccm_mle")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, initialize=0, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals, initialize=0, name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, initialize=0, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, initialize=0, name="y")
    model.abs_var = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, initialize=0, name="abs_val")
    
    model.obj = pyo.Objective(expr=sum( sum(offertimes[j] * data[i, j] * model.abs_var[i, j] for i in collection[j] ) for j in range(m)), sense=pyo.minimize)
    #model.obj = Objective(expr=-sum(data[i, j] * log(model.x[i, j]) for i in range(n) for j in range(m)),sense=minimize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    
    not_in_assortment = [[] for _ in range(m)]
    for j in range(m):
        for i in range(n):
            if i not in collection[j]:
                not_in_assortment[j].append(i)
    
    for j in range(m):
        for i in collection[j]:
            model.norm_constraint.add(model.x[i, j] == model.lam[i] + sum(model.y[k, j] * model.rho[k, i] for k in not_in_assortment[j]))
    for j in range(m):
        for i in not_in_assortment[j]:
            model.norm_constraint.add(model.y[i,j] == model.lam[i] + sum(model.y[k, i] * model.rho[k,i] for k in not_in_assortment[j]))
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] == 0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)

    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed

    #results = solver.solve(model, tee=True)
    
    #solver.options['max_iter'] = 10000  # Adjust max_iter as needed
    results = solver.solve(model, tee=True)
    status = str(results.solver.status)
    print("Pyomo optimization status:", status)
    
    # Access the optimal solution
    if status == pyo.SolverStatus.ok:
        #print('Optimal solution found!')
        model.write('model.mps', io_options={'symbolic_solver_labels': True})
        model.pprint()
    else:
        print('No solution found or an error occurred.')
    
    x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
    x_values = np.array(x_values)
    
    lam_values = [ model.lam[i].value for i in range(n)]
    lam_values = np.array(lam_values)
    
    rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
    rho_values = np.array(rho_values)
    
    # Retrieve the solver runtime
    runtime = results.solver.time
    print("Solver runtime:", runtime)
    
    #return [value(model.obj), x_values, results.solver.time]
    return [model.obj(), x_values, results.solver.time]
# Example usage:
# mccm_mle(data, collection, offer_times)


In [13]:
mccm_limit_loss_100, mccm_limit_prob_100,mccm_limit_runtime_100 = mccm_limit(choice_collection_100,collection_100,offer_times_100)
mccm_limit_loss_75, mccm_limit_prob_75,mccm_limit_runtime_75 = mccm_limit(choice_collection_75,collection_75,offer_times_75)
mccm_limit_loss_60, mccm_limit_prob_60,mccm_limit_runtime_60 = mccm_limit(choice_collection_60,collection_60,offer_times_60)
mccm_limit_loss_50, mccm_limit_prob_50,mccm_limit_runtime_50 = mccm_limit(choice_collection_50,collection_50,offer_times_50)
mccm_limit_loss_40, mccm_limit_prob_40,mccm_limit_runtime_40 = mccm_limit(choice_collection_40,collection_40,offer_times_40)
mccm_limit_loss_30, mccm_limit_prob_30,mccm_limit_runtime_30 = mccm_limit(choice_collection_30,collection_30,offer_times_30)


Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1801
Number of nonzeros in inequality constraint Jacobian.:      104
Number of nonzeros in Lagrangian Hessian.............:      228

Total number of variables............................:      387
                     variables with only lower bounds:      387
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

In [14]:
import pyomo.environ
from pyomo.environ import *

def mccm_limit2(data, collection, offertimes):
    #print(f'check the offertimes {offertimes}')
    model = ConcreteModel(name="mccm_mle")
    
    n, m = data.shape
    
    # add Variables
    model.lam = Var(range(n), within=NonNegativeReals, initialize=0, name="lam")
    model.rho = Var(range(n), range(n), within=NonNegativeReals, initialize=0, name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = Var(range(n), range(m), within=NonNegativeReals, initialize=0, name="x")
    model.y = Var(range(n), range(m), within=NonNegativeReals, initialize=0, name="y")
    model.abs_var = Var(range(n), range(m), within=NonNegativeReals, initialize=0, name="abs_val")
    
    model.obj = Objective(expr=sum( sum(offertimes[j] * data[i, j] * model.abs_var[i, j] for i in collection[j] ) for j in range(m)), sense=minimize)
    #model.obj = Objective(expr=-sum(data[i, j] * log(model.x[i, j]) for i in range(n) for j in range(m)),sense=minimize)
    
    model.norm_constraint = ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    
    not_in_assortment = [[] for _ in range(m)]
    for j in range(m):
        for i in range(n):
            if i not in collection[j]:
                not_in_assortment[j].append(i)
    
    for j in range(m):
        for i in collection[j]:
            model.norm_constraint.add(model.x[i, j] == model.lam[i] + sum(model.y[k, j] * model.rho[k, i] for k in not_in_assortment[j]))
    for j in range(m):
        for i in not_in_assortment[j]:
            model.norm_constraint.add(model.y[i,j] == model.lam[i] + sum(model.y[k, i] * model.rho[k,i] for k in not_in_assortment[j]))
    # constraint of the arrival rate
    model.arrival_rate_constraint = Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] == 0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)

    solver = SolverFactory('ipopt')  # You can change the solver as needed

    solver.options['expect_infeasible_problem'] = 'yes'  # or 'no' based on your assessment
    solver.options['bound_push'] = 1e-11
    solver.options['max_iter'] = 1000
    
    results = solver.solve(model, tee=True, logfile='solver.log')
    
    status = str(results.solver.status)
    print("Pyomo optimization status:", status)
    
    # Access the optimal solution
    if status == SolverStatus.ok:
        print('Optimal solution found!')
        #model.write('model.mps', io_options={'symbolic_solver_labels': True})
        #model.pprint()
    else:
        print('No solution found or an error occurred.')
    
    x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
    x_values = np.array(x_values)
    
    lam_values = [ model.lam[i].value for i in range(n)]
    lam_values = np.array(lam_values)
    
    rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
    rho_values = np.array(rho_values)
    
    # Retrieve the solver runtime
    runtime = results.solver.time
    print("Solver runtime:", runtime)
    
    #return [value(model.obj), x_values, results.solver.time]
    return [model.obj(), x_values, results.solver.time]


In [15]:
mccm_limit_loss_30, mccm_limit_prob_30,mccm_limit_runtime_30 = mccm_limit2(choice_collection_30,collection_30,offer_times_30)

Ipopt 3.12.12: expect_infeasible_problem=yes
bound_push=1e-11
max_iter=1000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3690
Number of nonzeros in inequality constraint Jacobian.:      256
Number of nonzeros in Lagrangian Hessian.............:      464

Total number of variables............................:      738
                     variables with only lower bounds:      738
                variables with lower and upper bounds:        0
      

In [16]:
mccm_limit_loss_20, mccm_limit_prob_20,mccm_limit_runtime_20 = mccm_limit2(choice_collection_20,collection_20,offer_times_20)

Ipopt 3.12.12: expect_infeasible_problem=yes
bound_push=1e-11
max_iter=1000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4392
Number of nonzeros in inequality constraint Jacobian.:      320
Number of nonzeros in Lagrangian Hessian.............:      556

Total number of variables............................:      873
                     variables with only lower bounds:      873
                variables with lower and upper bounds:        0
      

In [17]:
mccm_limit_loss = [mccm_limit_loss_20,mccm_limit_loss_30,mccm_limit_loss_40,mccm_limit_loss_50,mccm_limit_loss_60,mccm_limit_loss_75,mccm_limit_loss_100]
mccm_limit_prob = [mccm_limit_prob_20,mccm_limit_prob_30,mccm_limit_prob_40,mccm_limit_prob_50,mccm_limit_prob_60,mccm_limit_prob_75,mccm_limit_prob_100]
mccm_limit_runtime = [mccm_limit_runtime_20,mccm_limit_runtime_30,mccm_limit_runtime_40,mccm_limit_runtime_50,mccm_limit_runtime_60,mccm_limit_runtime_75,mccm_limit_runtime_100]
offertime_list = [20,30,40,50,60,75,100]
jd_mccm_limit_summary = pd.DataFrame({'offertimes':offertime_list,'mccm_loss':mccm_limit_loss,'mccm_limit_runtime':mccm_limit_runtime})
jd_mccm_limit_summary.to_csv('limit/summary/mccm_limit_summary.csv')

for i in range(len(offertime_list)):
    limit_prob = mccm_limit_prob[i]
    df_limit_prob = pd.DataFrame(limit_prob)
    df_limit_prob.to_csv('limit/probability/mccm/prob_offertimes'+str(offertime_list[i])+'.csv')

In [18]:
jd_mccm_limit_summary

,offertimes,mccm_loss,mccm_limit_runtime
0,20,218.294740,30.130095
1,30,197.536701,17.218676
2,40,166.094072,7.526283
3,50,83.972853,2.118375
4,60,68.136779,1.742772
5,75,57.894749,1.729044
6,100,50.391942,2.049280
